<a href="https://colab.research.google.com/github/Febisha/GenAI-Hackathon/blob/main/Gen_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Create Data
user_item_data = pd.DataFrame({
    'user_id': [1, 1, 2, 2, 3, 3, 4],
    'item_id': [101, 102, 101, 103, 104, 102, 105],
    'rating': [5, 3, 4, 5, 2, 4, 3]
})

item_metadata = pd.DataFrame({
    'item_id': [101, 102, 103, 104, 105],
    'category': ['Electronics', 'Clothing', 'Books', 'Electronics', 'Books']
})

# Step 2: User-Item Matrix
user_item_matrix = user_item_data.pivot(index='user_id', columns='item_id', values='rating').fillna(0)

# Step 3: Collaborative Filtering
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Step 4: Content-Based Filtering
item_categories = pd.get_dummies(item_metadata.set_index('item_id')['category'])
item_similarity = cosine_similarity(item_categories)
item_similarity_df = pd.DataFrame(item_similarity, index=item_categories.index, columns=item_categories.index)

# Step 5: Recommend Items
def recommend(user_id, top_n=3):
    # Collaborative Filtering
    similar_users = user_similarity_df.loc[user_id].sort_values(ascending=False).index[1:]
    collaborative_recs = set()
    for sim_user in similar_users:
        items = user_item_matrix.loc[sim_user]
        collaborative_recs.update(items[items > 0].index)

    # Content-Based Filtering
    user_items = user_item_matrix.loc[user_id]
    rated_items = user_items[user_items > 0].index
    content_recs = set()
    for item in rated_items:
        similar_items = item_similarity_df.loc[item].sort_values(ascending=False).index[1:top_n+1]
        content_recs.update(similar_items)

    # Combine recommendations and return
    final_recs = list(collaborative_recs.union(content_recs) - set(rated_items))
    return final_recs[:top_n]

# Step 6: Generate Recommendations for All Users
def generate_all_recommendations():
    all_recommendations = {}
    for user_id in user_item_matrix.index:
        all_recommendations[user_id] = recommend(user_id, top_n=5)
    return all_recommendations

# Print recommendations for all users
all_recs = generate_all_recommendations()
for user, recs in all_recs.items():
    print(f"Recommendations for User {user}: {recs}")


Recommendations for User 1: [104, 105, 103]
Recommendations for User 2: [104, 105, 102]
Recommendations for User 3: [105, 101, 103]
Recommendations for User 4: [104, 101, 102, 103]
